In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path("C:/Users/lakovskr/Desktop/portfolio/AplusA_PoC/data_raw")  # adapte si nécessaire

files = {
    "patients": DATA_DIR/"patients_survey.csv",
    "docs": DATA_DIR/"docs_survey.csv",
    "panel": DATA_DIR/"panel_longitudinal.csv"
}

# 1) Chargement
dfs = {k: pd.read_csv(path, parse_dates=True, dayfirst=False) for k, path in files.items()}

# 2) Basic info & counts
for name, df in dfs.items():
    print("=== Dataset:", name, "===")
    print("shape:", df.shape)
    print(df.dtypes)
    print("nulls per column:\n", df.isna().sum())
    print("unique counts (categorical candidates):")
    print(df.select_dtypes(include=['object','category']).nunique())
    print("\n--- head ---\n", df.head(5))
    print("\n\n")

# 3) Quick numeric stats and ranges
for name, df in dfs.items():
    nums = df.select_dtypes(include=[np.number])
    if not nums.empty:
        print("Numeric summary for", name)
        print(nums.describe().T[['count','mean','std','min','25%','50%','75%','max']])
    print("\n")

# 4) Spot-check correlations requested in spec
if 'ResponseScore' in dfs['patients'].columns and 'Adherence' in dfs['patients'].columns:
    print("Patients ResponseScore vs Adherence corr:", dfs['patients'][['ResponseScore','Adherence']].corr().iloc[0,1])

if 'YearsPractice' in dfs['docs'].columns and 'PreferBrand' in dfs['docs'].columns:
    # convert PreferBrand to 0/1 if needed
    d = dfs['docs'].copy()
    if d['PreferBrand'].dtype == object:
        d['PreferBrand_bin'] = d['PreferBrand'].map({'Yes':1,'No':0})
    else:
        d['PreferBrand_bin'] = d['PreferBrand']
    print("Docs YearsPractice vs PreferBrand corr:", d[['YearsPractice','PreferBrand_bin']].corr().iloc[0,1])

if 'FutureProb' in dfs['panel'].columns and 'FutureResponse' in dfs['panel'].columns:
    print("Panel FutureProb vs FutureResponse corr:", dfs['panel'][['FutureProb','FutureResponse']].corr().iloc[0,1])

# 5) Quick validation rules
# - exactly 1000 lines in patients and docs
assert dfs['patients'].shape[0] == 1000, "patients must have 1000 rows"
assert dfs['docs'].shape[0] == 1000, "docs must have 1000 rows"
assert dfs['panel'].shape[0] == 1000, "panel must have 1000 rows"

print("All basic shape checks passed (1000 rows each)")

# 6) Check patientID uniqueness (patients)
if 'PatientID' in dfs['patients'].columns:
    print("Unique PatientIDs (patients):", dfs['patients']['PatientID'].nunique())


=== Dataset: patients ===
shape: (1000, 10)
PatientID         object
Age                int64
Sex               object
Pathology         object
Treatment         object
ResponseScore    float64
Adherence        float64
Satisfaction       int64
Verbatim          object
SurveyDate        object
dtype: object
nulls per column:
 PatientID         0
Age               0
Sex               0
Pathology         0
Treatment         0
ResponseScore     0
Adherence        20
Satisfaction      0
Verbatim         50
SurveyDate        0
dtype: int64
unique counts (categorical candidates):
PatientID     1000
Sex              2
Pathology        5
Treatment        3
Verbatim       841
SurveyDate     316
dtype: int64

--- head ---
           PatientID  Age Sex  Pathology Treatment  ResponseScore  Adherence  \
0  2842de9d43a98b53   58   M  Oncologie         B           3.54       0.86   
1  fbeae7c18667b698   79   M     Asthme         B           2.49       0.92   
2  acbbb742c7524cec   34   F  Oncologie  

In [9]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path(r"C:/Users/lakovskr/Desktop/portfolio/AplusA_PoC/data_raw")
OUT_PARQUET = Path(r"C:/Users/lakovskr/Desktop/portfolio/AplusA_PoC/data_parquet")
OUT_PARQUET.mkdir(parents=True, exist_ok=True)

# load
patients = pd.read_csv(DATA_DIR/"patients_survey.csv")
docs     = pd.read_csv(DATA_DIR/"docs_survey.csv")
panel    = pd.read_csv(DATA_DIR/"panel_longitudinal.csv")

# 1) convert dates
patients['SurveyDate'] = pd.to_datetime(patients['SurveyDate'], errors='coerce')
panel['VisitDate'] = pd.to_datetime(panel['VisitDate'], errors='coerce')

# 2) missing flags
patients['Adherence_missing'] = patients['Adherence'].isna().astype(int)
patients['Verbatim_missing'] = patients['Verbatim'].isna().astype(int)
docs['Comment_missing'] = docs['Comment'].isna().astype(int)
panel['Biomarker_missing'] = panel['Biomarker'].isna().astype(int)

# 3) impute numeric missing
patients['Adherence'] = patients['Adherence'].fillna(patients['Adherence'].median())
global_biom_median = panel['Biomarker'].median()
panel['Biomarker'] = panel.groupby('PatientID')['Biomarker'].transform(lambda g: g.fillna(g.median()))
panel['Biomarker'] = panel['Biomarker'].fillna(global_biom_median)

# 4) Verbatim: fill placeholder if any left
patients['Verbatim'] = patients['Verbatim'].fillna("No comment")

# 5) Visits per patient summary (useful feature)
visits_per_patient = panel.groupby('PatientID').size().rename('n_visits')
print("Visits distribution:\n", visits_per_patient.value_counts().sort_index())

# 6) Check class balance for panel target
print("FutureResponse balance:\n", panel['FutureResponse'].value_counts(normalize=True).to_dict())
print("FutureProb mean:", panel['FutureProb'].mean())

# 7) Save cleaned CSV + parquet
patients.to_csv(DATA_DIR/"patients_survey_clean.csv", index=False)
docs.to_csv(DATA_DIR/"docs_survey_clean.csv", index=False)
panel.to_csv(DATA_DIR/"panel_longitudinal_clean.csv", index=False)

patients.to_parquet(OUT_PARQUET/"patients_survey.parquet", index=False)
docs.to_parquet(OUT_PARQUET/"docs_survey.parquet", index=False)
panel.to_parquet(OUT_PARQUET/"panel_longitudinal.parquet", index=False)

print("Saved cleaned CSVs and parquet.")


C:\Users\lakovskr\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Visits distribution:
 n_visits
1    106
2    113
3     88
4     61
5     32
Name: count, dtype: int64
FutureResponse balance:
 {0: 0.872, 1: 0.128}
FutureProb mean: 0.31524699999999994
Saved cleaned CSVs and parquet.


In [10]:
from pathlib import Path
import pandas as pd

base = Path(r"C:/Users/lakovskr/Desktop/portfolio/AplusA_PoC")
data_raw = base / "data_raw"
silver = base / "silver"
silver.mkdir(exist_ok=True)

# List the cleaned CSVs you want to convert
csv_files = [
    "patients_survey_clean.csv",
    "docs_survey_clean.csv",
    "panel_longitudinal_clean.csv"
]


In [11]:
for fname in csv_files:
    csv_path = data_raw / fname
    df = pd.read_csv(csv_path)
    parquet_path = silver / fname.replace("_clean.csv", "_clean.parquet")
    df.to_parquet(parquet_path, index=False)
    print(f"Saved {parquet_path}")

Saved C:\Users\lakovskr\Desktop\portfolio\AplusA_PoC\silver\patients_survey_clean.parquet
Saved C:\Users\lakovskr\Desktop\portfolio\AplusA_PoC\silver\docs_survey_clean.parquet
Saved C:\Users\lakovskr\Desktop\portfolio\AplusA_PoC\silver\panel_longitudinal_clean.parquet


In [12]:
for f in silver.glob("*.parquet"):
    print(f.name, "->", pd.read_parquet(f).shape)

docs_survey_clean.parquet -> (1000, 8)
panel_longitudinal_clean.parquet -> (1000, 10)
patients_survey_clean.parquet -> (1000, 12)


In [15]:
"""
Build GOLD layer from SILVER parquet files
Creates analytical models ready for BI/ML consumption
"""
import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================================
# CONFIGURATION
# ============================================================================
BASE = Path(r"C:/Users/lakovskr/Desktop/portfolio/AplusA_PoC")
SILVER = BASE / "silver"
GOLD = BASE / "gold"
GOLD.mkdir(parents=True, exist_ok=True)

# ============================================================================
# LOAD SILVER TABLES
# ============================================================================
print("Loading SILVER layer...")
patients = pd.read_parquet(SILVER / "patients_survey_clean.parquet")
docs = pd.read_parquet(SILVER / "docs_survey_clean.parquet")
panel = pd.read_parquet(SILVER / "panel_longitudinal_clean.parquet")

print(f"  patients: {patients.shape}")
print(f"  docs: {docs.shape}")
print(f"  panel: {panel.shape}")

# ============================================================================
# DATA TYPE ENFORCEMENT
# ============================================================================
print("\nEnforcing data types...")
patients['SurveyDate'] = pd.to_datetime(patients['SurveyDate'], errors='coerce')
panel['VisitDate'] = pd.to_datetime(panel['VisitDate'], errors='coerce')
patients['PatientID'] = patients['PatientID'].astype(str)
panel['PatientID'] = panel['PatientID'].astype(str)
docs['DocID'] = docs['DocID'].astype(str)

# ============================================================================
# BUILD PANEL AGGREGATES (Patient-level features from visits)
# ============================================================================
print("\nAggregating panel data to patient level...")
panel_agg = panel.groupby('PatientID').agg(
    n_visits=('VisitDate', 'count'),
    first_visit=('VisitDate', 'min'),
    last_visit=('VisitDate', 'max'),
    biomarker_mean=('Biomarker', 'mean'),
    biomarker_std=('Biomarker', 'std'),
    biomarker_min=('Biomarker', 'min'),
    biomarker_max=('Biomarker', 'max'),
    response_mean=('ResponseScore', 'mean'),
    response_std=('ResponseScore', 'std'),
    future_resp_rate=('FutureResponse', 'mean'),
    future_prob_mean=('FutureProb', 'mean'),
    event_count=('EventFlag', 'sum'),
    event_rate=('EventFlag', 'mean')
).reset_index()

# Fill NaN std with 0 (single visit cases)
panel_agg['biomarker_std'] = panel_agg['biomarker_std'].fillna(0)
panel_agg['response_std'] = panel_agg['response_std'].fillna(0)

# Recency: days since last visit
ref_date = panel['VisitDate'].max()
panel_agg['days_since_last_visit'] = (ref_date - panel_agg['last_visit']).dt.days

# Visit frequency: visits per month active
panel_agg['months_active'] = ((panel_agg['last_visit'] - panel_agg['first_visit']).dt.days / 30.44).clip(lower=1)
panel_agg['visits_per_month'] = panel_agg['n_visits'] / panel_agg['months_active']

print(f"  panel_agg: {panel_agg.shape}")

# ============================================================================
# BUILD PATIENTS_FEATURES (Main analytical table)
# ============================================================================
print("\nBuilding patients_features table...")
patients_features = patients.merge(panel_agg, on='PatientID', how='left')

# Age buckets
patients_features['age_bucket'] = pd.cut(
    patients_features['Age'],
    bins=[17, 30, 45, 60, 75, 120],
    labels=['18-30', '31-45', '46-60', '61-75', '76+']
)

# Treatment numeric encoding
treatment_map = {'A': 0, 'B': 1, 'C': 2}
patients_features['treatment_num'] = patients_features['Treatment'].map(treatment_map)

# Binary flags
patients_features['satisfied'] = (patients_features['Satisfaction'] >= 4).astype(int)
patients_features['high_adherence'] = (patients_features['Adherence'] >= 0.8).astype(int)
patients_features['high_responder'] = (patients_features['ResponseScore'] >= 7).astype(int)

# Adherence categories
patients_features['adherence_cat'] = pd.cut(
    patients_features['Adherence'],
    bins=[-0.01, 0.6, 0.8, 0.95, 1.01],
    labels=['<60%', '60-80%', '80-95%', '95-100%']
).astype(str)

# Response categories
patients_features['response_cat'] = pd.cut(
    patients_features['ResponseScore'],
    bins=[-0.01, 4, 7, 10.01],
    labels=['Low (0-4)', 'Medium (4-7)', 'High (7-10)']
).astype(str)

# Fill NaNs for patients without panel visits
fill_dict = {
    'n_visits': 0,
    'event_count': 0,
    'event_rate': 0,
    'future_resp_rate': 0,
    'future_prob_mean': 0,
    'days_since_last_visit': 999,
    'visits_per_month': 0,
    'months_active': 0
}
patients_features = patients_features.fillna(fill_dict)

# Impute biomarker/response means with survey baseline
patients_features['biomarker_mean'] = patients_features['biomarker_mean'].fillna(
    patients_features['biomarker_mean'].median()
)
patients_features['response_mean'] = patients_features['response_mean'].fillna(
    patients_features['ResponseScore']
)

# Convert numeric types
int_cols = ['n_visits', 'event_count', 'days_since_last_visit']
for col in int_cols:
    patients_features[col] = patients_features[col].astype(int)

print(f"  patients_features: {patients_features.shape}")

# ============================================================================
# BUILD DOCS_SUMMARY_AGG (Specialty-level aggregates)
# ============================================================================
print("\nBuilding docs_summary_agg table...")
docs_summary_agg = docs.groupby('Specialty').agg(
    n_docs=('DocID', 'count'),
    avg_years_practice=('YearsPractice', 'mean'),
    median_years_practice=('YearsPractice', 'median'),
    pct_prefer_brand=('PreferBrand', lambda x: (x == 'Yes').sum() / x.count()),
    avg_criteria_score=('CriteriaScore', 'mean'),
    pct_high_freq=('PrescriptionFreq', lambda x: (x == 'High').sum() / x.count())
).reset_index()

# Round percentages
docs_summary_agg['pct_prefer_brand'] = (docs_summary_agg['pct_prefer_brand'] * 100).round(1)
docs_summary_agg['pct_high_freq'] = (docs_summary_agg['pct_high_freq'] * 100).round(1)

print(f"  docs_summary_agg: {docs_summary_agg.shape}")

# ============================================================================
# BUILD PATIENT_PANEL_JOINED (Visit-level analytical table)
# ============================================================================
print("\nBuilding patient_panel_joined table...")
# NOTE: Using explicit suffixes to handle ResponseScore column collision
# - ResponseScore_visit: actual response at each visit (from panel)
# - ResponseScore_baseline: initial survey response (from patients)
patient_panel_joined = panel.merge(
    patients[['PatientID', 'Age', 'Sex', 'Pathology', 'Treatment', 
              'Adherence', 'Satisfaction', 'ResponseScore']],
    on='PatientID',
    how='left',
    validate='m:1',
    suffixes=('_visit', '_baseline')
)

# Add first visit per patient
first_visit = panel.groupby('PatientID')['VisitDate'].min().rename('first_visit').reset_index()
patient_panel_joined = patient_panel_joined.merge(first_visit, on='PatientID', how='left')

# Days from first visit
patient_panel_joined['days_from_first_visit'] = (
    patient_panel_joined['VisitDate'] - patient_panel_joined['first_visit']
).dt.days

# Visit sequence number
patient_panel_joined = patient_panel_joined.sort_values(['PatientID', 'VisitDate'])
patient_panel_joined['visit_number'] = patient_panel_joined.groupby('PatientID').cumcount() + 1

# Age bucket
patient_panel_joined['age_bucket'] = pd.cut(
    patient_panel_joined['Age'],
    bins=[17, 30, 45, 60, 75, 120],
    labels=['18-30', '31-45', '46-60', '61-75', '76+']
)

print(f"  patient_panel_joined: {patient_panel_joined.shape}")

# ============================================================================
# SAVE GOLD TABLES
# ============================================================================
print("\nSaving GOLD tables...")

# Parquet (compressed, best for analytics)
patients_features.to_parquet(GOLD / "patients_features.parquet", index=False)
docs.to_parquet(GOLD / "docs_survey.parquet", index=False)  # Keep full table
docs_summary_agg.to_parquet(GOLD / "docs_summary_agg.parquet", index=False)
patient_panel_joined.to_parquet(GOLD / "patient_panel_joined.parquet", index=False)

# CSV (for compatibility)
patients_features.to_csv(GOLD / "patients_features.csv", index=False)
docs.to_csv(GOLD / "docs_survey.csv", index=False)
docs_summary_agg.to_csv(GOLD / "docs_summary_agg.csv", index=False)
patient_panel_joined.to_csv(GOLD / "patient_panel_joined.csv", index=False)

print("✓ patients_features:", patients_features.shape)
print("✓ docs_survey:", docs.shape)
print("✓ docs_summary_agg:", docs_summary_agg.shape)
print("✓ patient_panel_joined:", patient_panel_joined.shape)

# ============================================================================
# VALIDATION OUTPUTS
# ============================================================================
print("\n" + "=" * 80)
print("VALIDATION: PATIENTS_FEATURES")
print("=" * 80)
print("\nColumns:", patients_features.columns.tolist())
print("\nSample rows:")
print(patients_features.head(3)[['PatientID', 'Age', 'Sex', 'Treatment', 'Adherence', 
                                   'n_visits', 'response_mean', 'adherence_cat']].to_string())
print("\nNumerical Summary:")
print(patients_features[['Age', 'Adherence', 'Satisfaction', 'n_visits', 
                          'biomarker_mean', 'response_mean']].describe().round(2))

print("\n" + "=" * 80)
print("VALIDATION: DOCS_SUMMARY_AGG")
print("=" * 80)
print(docs_summary_agg.to_string(index=False))

print("\n" + "=" * 80)
print("VALIDATION: PATIENT_PANEL_JOINED")
print("=" * 80)
print("\nColumns:", patient_panel_joined.columns.tolist())
print("\nSample rows:")
print(patient_panel_joined.head(5)[['PatientID', 'VisitDate', 'visit_number', 
                                     'Biomarker', 'ResponseScore_visit', 'EventFlag']].to_string())

print("\n" + "=" * 80)
print("GOLD LAYER BUILD COMPLETE")
print("=" * 80)

Loading SILVER layer...
  patients: (1000, 12)
  docs: (1000, 8)
  panel: (1000, 10)

Enforcing data types...

Aggregating panel data to patient level...
  panel_agg: (400, 17)

Building patients_features table...
  patients_features: (1000, 35)

Building docs_summary_agg table...
  docs_summary_agg: (5, 7)

Building patient_panel_joined table...
  patient_panel_joined: (1000, 21)

Saving GOLD tables...
✓ patients_features: (1000, 35)
✓ docs_survey: (1000, 8)
✓ docs_summary_agg: (5, 7)
✓ patient_panel_joined: (1000, 21)

VALIDATION: PATIENTS_FEATURES

Columns: ['PatientID', 'Age', 'Sex', 'Pathology', 'Treatment', 'ResponseScore', 'Adherence', 'Satisfaction', 'Verbatim', 'SurveyDate', 'Adherence_missing', 'Verbatim_missing', 'n_visits', 'first_visit', 'last_visit', 'biomarker_mean', 'biomarker_std', 'biomarker_min', 'biomarker_max', 'response_mean', 'response_std', 'future_resp_rate', 'future_prob_mean', 'event_count', 'event_rate', 'days_since_last_visit', 'months_active', 'visits_per_

In [16]:
"""
ML Training & Comparison Pipeline - FIXED VERSION
Trains multiple models to predict patient future response
Handles categorical features, provides SHAP explanations, saves all outputs
"""
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, 
                             precision_recall_curve, precision_score, 
                             recall_score, f1_score, brier_score_loss,
                             roc_curve)
import joblib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# ============================================================================
# CONFIGURATION
# ============================================================================
BASE = Path(r"C:/Users/lakovskr/Desktop/portfolio/AplusA_PoC")
GOLD = BASE / "gold"
OUT = GOLD  # save outputs into gold

print("=" * 80)
print("ML TRAINING PIPELINE - PATIENT RESPONSE PREDICTION")
print("=" * 80)

# ============================================================================
# LOAD DATA
# ============================================================================
print("\nLoading gold table...")
df = pd.read_parquet(GOLD / "patients_features.parquet")
print(f"  Loaded: {df.shape}")

# ============================================================================
# TARGET CREATION
# ============================================================================
print("\nCreating target variable...")
# Option 1: Binary target from future_resp_rate
if 'future_resp_rate' in df.columns:
    # Check different thresholds
    print("  Checking target balance at different thresholds:")
    for thresh in [0, 0.3, 0.5]:
        pct = (df['future_resp_rate'] > thresh).mean()
        print(f"    future_resp_rate > {thresh}: {pct:.1%} positive")
    
    # Use threshold 0 (any future response)
    df['y'] = (df['future_resp_rate'] > 0).astype(int)
    print(f"\n  ✓ Using: future_resp_rate > 0")
    print(f"    Class balance: {df['y'].value_counts(normalize=True).to_dict()}")
else:
    raise ValueError("No future_resp_rate in patients_features")

# ============================================================================
# FEATURE ENGINEERING
# ============================================================================
print("\nPreparing features...")

# Drop non-feature columns (FIXED: removed ResponseScore_baseline - doesn't exist here)
drop_cols = [
    'PatientID', 'Verbatim', 'SurveyDate', 
    'first_visit', 'last_visit', 'age_bucket',
    'y', 'future_resp_rate', 'future_prob_mean',  # target & related
    'ResponseScore'  # Already have response_mean from panel
]

candidate_features = [c for c in df.columns if c not in drop_cols]
print(f"  Candidate features: {len(candidate_features)}")

# Create feature matrix
X = df[candidate_features].copy()

# Handle categorical features (FIXED: encode instead of dropping)
print("\nEncoding categorical features...")
categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
if categorical_cols:
    print(f"  Categorical columns: {categorical_cols}")
    le = LabelEncoder()
    for col in categorical_cols:
        if col in X.columns:
            X[col] = le.fit_transform(X[col].astype(str))
            print(f"    ✓ Encoded: {col}")

# Convert to numeric and handle missing values (FIXED: use median instead of 0)
X = X.select_dtypes(include=[np.number])
print(f"\n  Final feature count: {X.shape[1]}")

# Check missing values
missing = X.isnull().sum()
if missing.sum() > 0:
    print(f"\n  Handling missing values:")
    print(f"    Features with NaN: {missing[missing > 0].to_dict()}")
    X = X.fillna(X.median())
    print(f"    ✓ Filled with median")
else:
    print(f"\n  ✓ No missing values")

# Get target
y = df['y'].astype(int)

print(f"\nFinal dataset: {X.shape}")
print(f"Features used: {X.columns.tolist()}")
print(f"Class balance: {y.value_counts(normalize=True).round(3).to_dict()}")

# ============================================================================
# TRAIN-TEST SPLIT
# ============================================================================
print("\nSplitting data...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)
print(f"  Train: {X_train.shape}")
print(f"  Test: {X_test.shape}")

# ============================================================================
# DEFINE MODELS
# ============================================================================
print("\nSetting up models...")
models = {
    'logreg': Pipeline([
        ('scaler', StandardScaler()), 
        ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42))
    ]),
    'rf': Pipeline([
        ('clf', RandomForestClassifier(n_estimators=200, max_depth=10, 
                                      random_state=42, class_weight='balanced'))
    ]),
}

# Try to add XGBoost
try:
    import xgboost as xgb
    models['xgb'] = Pipeline([
        ('clf', xgb.XGBClassifier(
            use_label_encoder=False, 
            eval_metric='logloss', 
            random_state=42,
            max_depth=6,
            n_estimators=200
        ))
    ])
    print("  ✓ XGBoost available")
except:
    print("  ⚠ XGBoost not available")
    # Try LightGBM as fallback
    try:
        import lightgbm as lgb
        models['lgbm'] = Pipeline([
            ('clf', lgb.LGBMClassifier(random_state=42, n_estimators=200))
        ])
        print("  ✓ LightGBM available")
    except:
        print("  ⚠ LightGBM not available")

print(f"\n  Models to train: {list(models.keys())}")

# ============================================================================
# TRAIN & EVALUATE MODELS
# ============================================================================
print("\n" + "=" * 80)
print("TRAINING MODELS")
print("=" * 80)

results = []

for name, pipe in models.items():
    print(f"\n[{name.upper()}]")
    print("-" * 40)
    
    # Train
    pipe.fit(X_train, y_train)
    
    # Predict
    proba = pipe.predict_proba(X_test)[:, 1]
    pred = (proba >= 0.5).astype(int)
    
    # Metrics
    auc = roc_auc_score(y_test, proba)
    prauc = average_precision_score(y_test, proba)
    prec = precision_score(y_test, pred, zero_division=0)
    rec = recall_score(y_test, pred, zero_division=0)
    f1 = f1_score(y_test, pred, zero_division=0)
    brier = brier_score_loss(y_test, proba)
    
    results.append({
        'model': name, 
        'auc': auc, 
        'pr_auc': prauc, 
        'precision': prec, 
        'recall': rec, 
        'f1': f1, 
        'brier': brier
    })
    
    print(f"  AUC:       {auc:.3f}")
    print(f"  PR-AUC:    {prauc:.3f}")
    print(f"  Precision: {prec:.3f}")
    print(f"  Recall:    {rec:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Brier:     {brier:.3f}")
    
    # Save model
    joblib.dump(pipe, OUT / f"model_{name}.joblib")
    print(f"  ✓ Saved: model_{name}.joblib")
    
    # Save predictions
    preds_df = pd.DataFrame({
        'PatientID': df.loc[X_test.index, 'PatientID'],
        'y_true': y_test.values,
        'y_proba': proba,
        'y_pred': pred
    })
    preds_df.to_csv(OUT / f"predictions_{name}.csv", index=False)

# ============================================================================
# SAVE MODEL COMPARISON
# ============================================================================
print("\n" + "=" * 80)
print("MODEL COMPARISON")
print("=" * 80)

scores_df = pd.DataFrame(results).sort_values('auc', ascending=False)
print("\n" + scores_df.to_string(index=False))

scores_df.to_csv(OUT / "model_scores.csv", index=False)
print(f"\n✓ Saved: model_scores.csv")

# Select best model
best_name = scores_df.iloc[0]['model']
best_model = joblib.load(OUT / f"model_{best_name}.joblib")
print(f"\n🏆 Best model: {best_name.upper()} (AUC={scores_df.iloc[0]['auc']:.3f})")

# ============================================================================
# FEATURE IMPORTANCE (for tree models)
# ============================================================================
if best_name in ['rf', 'xgb', 'lgbm']:
    print("\n" + "=" * 80)
    print("FEATURE IMPORTANCE")
    print("=" * 80)
    
    estimator = best_model.named_steps['clf']
    
    # Get feature importance
    importance = pd.DataFrame({
        'feature': X.columns,
        'importance': estimator.feature_importances_
    }).sort_values('importance', ascending=False)
    
    importance.to_csv(OUT / f"feature_importance_{best_name}.csv", index=False)
    print(f"\n✓ Saved: feature_importance_{best_name}.csv")
    
    print("\nTop 10 features:")
    print(importance.head(10).to_string(index=False))
    
    # Plot
    plt.figure(figsize=(10, 6))
    top_n = min(15, len(importance))
    plt.barh(range(top_n), importance['importance'].head(top_n))
    plt.yticks(range(top_n), importance['feature'].head(top_n))
    plt.xlabel('Importance')
    plt.title(f'Top {top_n} Features - {best_name.upper()}')
    plt.tight_layout()
    plt.savefig(OUT / f"feature_importance_{best_name}.png", dpi=150)
    plt.close()
    print(f"✓ Saved: feature_importance_{best_name}.png")

# ============================================================================
# SHAP EXPLANATIONS (FIXED)
# ============================================================================
print("\n" + "=" * 80)
print("SHAP EXPLANATIONS")
print("=" * 80)

try:
    import shap
    
    print(f"\nComputing SHAP values for {best_name}...")
    
    if best_name in ['rf', 'xgb', 'lgbm']:
        # Tree models: use TreeExplainer (fast)
        estimator = best_model.named_steps.get('clf', best_model)
        explainer = shap.TreeExplainer(estimator)
        shap_values = explainer.shap_values(X_test)
        
        # Handle multiclass output (take positive class)
        if isinstance(shap_values, list):
            sv = shap_values[1]
        else:
            sv = shap_values
        
        # Summary plot
        shap.summary_plot(sv, X_test, show=False, max_display=20)
        plt.tight_layout()
        plt.savefig(OUT / f"shap_summary_{best_name}.png", dpi=150, bbox_inches='tight')
        plt.close()
        print(f"✓ Saved: shap_summary_{best_name}.png")
        
        # Dot plot (FIXED: use summary_plot with plot_type)
        shap.summary_plot(sv, X_test, plot_type="dot", show=False, max_display=20)
        plt.tight_layout()
        plt.savefig(OUT / f"shap_dot_{best_name}.png", dpi=150, bbox_inches='tight')
        plt.close()
        print(f"✓ Saved: shap_dot_{best_name}.png")
        
    else:
        # Logistic Regression: use LinearExplainer (faster than Kernel)
        print("  Using LinearExplainer for logistic regression...")
        # Get the actual LR model and scaler
        scaler = best_model.named_steps.get('scaler')
        clf = best_model.named_steps.get('clf')
        
        # Transform data
        X_test_scaled = scaler.transform(X_test)
        
        # Create explainer
        explainer = shap.LinearExplainer(clf, X_test_scaled)
        shap_values = explainer.shap_values(X_test_scaled)
        
        # Summary plot
        shap.summary_plot(shap_values, X_test, show=False, max_display=20)
        plt.tight_layout()
        plt.savefig(OUT / f"shap_summary_{best_name}.png", dpi=150, bbox_inches='tight')
        plt.close()
        print(f"✓ Saved: shap_summary_{best_name}.png")
        
except Exception as e:
    print(f"⚠ SHAP computation failed: {e}")
    print("  Continuing without SHAP explanations...")

# ============================================================================
# ROC & PR CURVES
# ============================================================================
print("\n" + "=" * 80)
print("PERFORMANCE CURVES")
print("=" * 80)

proba_best = best_model.predict_proba(X_test)[:, 1]

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, proba_best)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f"AUC = {roc_auc_score(y_test, proba_best):.3f}")
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random')
plt.xlabel("False Positive Rate", fontsize=12)
plt.ylabel("True Positive Rate", fontsize=12)
plt.title(f"ROC Curve - {best_name.upper()}", fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(OUT / "roc_curve_best.png", dpi=150)
plt.close()
print("✓ Saved: roc_curve_best.png")

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, proba_best)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, linewidth=2, 
         label=f"PR-AUC = {average_precision_score(y_test, proba_best):.3f}")
plt.axhline(y=y_test.mean(), color='k', linestyle='--', alpha=0.5, 
            label=f'Baseline = {y_test.mean():.3f}')
plt.xlabel("Recall", fontsize=12)
plt.ylabel("Precision", fontsize=12)
plt.title(f"Precision-Recall Curve - {best_name.upper()}", fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(OUT / "pr_curve_best.png", dpi=150)
plt.close()
print("✓ Saved: pr_curve_best.png")

# ============================================================================
# FULL DATASET PREDICTIONS
# ============================================================================
print("\n" + "=" * 80)
print("GENERATING FULL DATASET PREDICTIONS")
print("=" * 80)

full_proba = best_model.predict_proba(X)[:, 1]
full_pred = (full_proba >= 0.5).astype(int)

out_full = pd.DataFrame({
    'PatientID': df['PatientID'],
    'y_true': df['y'],
    'y_proba': full_proba,
    'y_pred': full_pred,
    'risk_category': pd.cut(full_proba, bins=[0, 0.3, 0.7, 1.0], 
                            labels=['Low', 'Medium', 'High'])
})

out_full.to_csv(OUT / "predictions_full_bestmodel.csv", index=False)
print(f"✓ Saved: predictions_full_bestmodel.csv ({len(out_full)} patients)")

print("\nRisk distribution:")
print(out_full['risk_category'].value_counts().sort_index())

# ============================================================================
# FINAL REPORT
# ============================================================================
print("\n" + "=" * 80)
print("GENERATING REPORT")
print("=" * 80)

with open(OUT / "model_report.txt", "w") as f:
    f.write("=" * 80 + "\n")
    f.write("ML MODEL TRAINING REPORT - PATIENT RESPONSE PREDICTION\n")
    f.write("=" * 80 + "\n\n")
    
    f.write(f"Best Model: {best_name.upper()}\n")
    f.write(f"AUC: {scores_df.iloc[0]['auc']:.3f}\n")
    f.write(f"PR-AUC: {scores_df.iloc[0]['pr_auc']:.3f}\n\n")
    
    f.write("=" * 80 + "\n")
    f.write("MODEL COMPARISON\n")
    f.write("=" * 80 + "\n")
    f.write(scores_df.to_string(index=False) + "\n\n")
    
    f.write("=" * 80 + "\n")
    f.write("DATASET INFO\n")
    f.write("=" * 80 + "\n")
    f.write(f"Total patients: {len(df)}\n")
    f.write(f"Features used: {X.shape[1]}\n")
    f.write(f"Class balance: {y.value_counts(normalize=True).to_dict()}\n\n")
    
    f.write("=" * 80 + "\n")
    f.write("RECOMMENDATIONS\n")
    f.write("=" * 80 + "\n")
    f.write("1. Validate threshold (0.5) against business objectives\n")
    f.write("2. Consider calibration for probability interpretation\n")
    f.write("3. Monitor model performance on new data\n")
    f.write("4. Investigate misclassified high-risk patients\n")
    f.write("5. Consider temporal validation (train on older data, test on recent)\n")

print("✓ Saved: model_report.txt")

# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "=" * 80)
print("PIPELINE COMPLETE")
print("=" * 80)
print(f"\n📂 All outputs saved to: {OUT}")
print("\nFiles created:")
print(f"  • model_*.joblib ({len(models)} models)")
print(f"  • predictions_*.csv ({len(models)} + 1 files)")
print(f"  • model_scores.csv")
print(f"  • feature_importance_*.csv/.png")
print(f"  • shap_summary_*.png")
print(f"  • roc_curve_best.png")
print(f"  • pr_curve_best.png")
print(f"  • model_report.txt")
print("\n🏆 Best model:", best_name.upper())
print(f"   AUC: {scores_df.iloc[0]['auc']:.3f}")
print(f"   Ready for deployment!")
print("\n" + "=" * 80)

ML TRAINING PIPELINE - PATIENT RESPONSE PREDICTION

Loading gold table...
  Loaded: (1000, 35)

Creating target variable...
  Checking target balance at different thresholds:
    future_resp_rate > 0: 11.1% positive
    future_resp_rate > 0.3: 7.5% positive
    future_resp_rate > 0.5: 2.0% positive

  ✓ Using: future_resp_rate > 0
    Class balance: {0: 0.889, 1: 0.111}

Preparing features...
  Candidate features: 26

Encoding categorical features...
  Categorical columns: ['Sex', 'Pathology', 'Treatment', 'adherence_cat', 'response_cat']
    ✓ Encoded: Sex
    ✓ Encoded: Pathology
    ✓ Encoded: Treatment
    ✓ Encoded: adherence_cat
    ✓ Encoded: response_cat

  Final feature count: 26

  Handling missing values:
    Features with NaN: {'biomarker_std': 600, 'biomarker_min': 600, 'biomarker_max': 600, 'response_std': 600}
    ✓ Filled with median

Final dataset: (1000, 26)
Features used: ['Age', 'Sex', 'Pathology', 'Treatment', 'Adherence', 'Satisfaction', 'Adherence_missing', 'Verb

In [ ]:
"""
04_production_artifacts.py
Generate calibration, optimal thresholds, SHAP explanations for Power BI
Run AFTER 03_ml_train_compare.py
"""
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import matplotlib.pyplot as plt
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import precision_recall_curve, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# ============================================================================
# CONFIGURATION
# ============================================================================
BASE = Path(r"C:/Users/lakovskr/Desktop/portfolio/AplusA_PoC")
GOLD = BASE / "gold"
OUT = GOLD

print("=" * 80)
print("PRODUCTION ARTIFACTS - CALIBRATION, THRESHOLDS, SHAP")
print("=" * 80)

# ============================================================================
# LOAD ARTIFACTS FROM STEP 3
# ============================================================================
print("\nLoading trained models and data...")
df = pd.read_parquet(GOLD / "patients_features.parquet")
predictions = pd.read_csv(GOLD / "predictions_full_bestmodel.csv")

# Load best model
best_model = joblib.load(GOLD / "model_rf.joblib")
print(f"✓ Loaded RandomForest model")

# Recreate X, y (same as training)
drop_cols = [
    'PatientID', 'Verbatim', 'SurveyDate', 
    'first_visit', 'last_visit', 'age_bucket',
    'future_resp_rate', 'future_prob_mean', 'ResponseScore'
]
candidate_features = [c for c in df.columns if c not in drop_cols]
X = df[candidate_features].copy()

# Encode categoricals
from sklearn.preprocessing import LabelEncoder
categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
if categorical_cols:
    le = LabelEncoder()
    for col in categorical_cols:
        X[col] = le.fit_transform(X[col].astype(str))

X = X.select_dtypes(include=[np.number]).fillna(X.median())
y = (df['future_resp_rate'] > 0).astype(int)

# Train-test split (same random state as training)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# ============================================================================
# A. CALIBRATION (Platt Scaling)
# ============================================================================
print("\n" + "=" * 80)
print("A. PROBABILITY CALIBRATION (Platt Scaling)")
print("=" * 80)

# Calibrate on test set
print("\nCalibrating model probabilities...")
calibrated_model = CalibratedClassifierCV(best_model, method='sigmoid', cv='prefit')
calibrated_model.fit(X_test, y_test)

# Get calibrated probabilities for full dataset
proba_calibrated = calibrated_model.predict_proba(X)[:, 1]

# Save calibrated model
joblib.dump(calibrated_model, OUT / "model_rf_calibrated.joblib")
print("✓ Saved: model_rf_calibrated.joblib")

# Update predictions with calibrated probs
predictions['y_proba_calibrated'] = proba_calibrated
predictions.to_csv(OUT / "predictions_full_calibrated.csv", index=False)
print("✓ Saved: predictions_full_calibrated.csv")

# Calibration curve comparison
proba_test = best_model.predict_proba(X_test)[:, 1]
proba_test_calib = calibrated_model.predict_proba(X_test)[:, 1]

fraction_positives_orig, mean_predicted_orig = calibration_curve(
    y_test, proba_test, n_bins=10, strategy='quantile'
)
fraction_positives_calib, mean_predicted_calib = calibration_curve(
    y_test, proba_test_calib, n_bins=10, strategy='quantile'
)

# Plot calibration
plt.figure(figsize=(10, 6))
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect calibration')
plt.plot(mean_predicted_orig, fraction_positives_orig, 's-', 
         label='Original RF', linewidth=2, markersize=8)
plt.plot(mean_predicted_calib, fraction_positives_calib, 'o-', 
         label='Calibrated RF (Platt)', linewidth=2, markersize=8)
plt.xlabel('Mean Predicted Probability', fontsize=12)
plt.ylabel('Fraction of Positives', fontsize=12)
plt.title('Calibration Curve Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(OUT / "calibration_curve.png", dpi=150)
plt.close()
print("✓ Saved: calibration_curve.png")

print("\nCalibration metrics:")
print(f"  Original Brier score: {((proba_test - y_test)**2).mean():.4f}")
print(f"  Calibrated Brier score: {((proba_test_calib - y_test)**2).mean():.4f}")

# ============================================================================
# B. OPTIMAL THRESHOLDS (Two Strategies)
# ============================================================================
print("\n" + "=" * 80)
print("B. THRESHOLD OPTIMIZATION")
print("=" * 80)

# Use calibrated probabilities for threshold selection
proba_test_final = proba_test_calib

# Strategy 1: High Sensitivity (Recall-oriented - catch all positives)
print("\nStrategy 1: HIGH SENSITIVITY (Safety/Monitoring)")
print("  Goal: Maximize recall, accept lower precision")

# Find threshold that gives 90%+ recall
precision, recall, thresholds = precision_recall_curve(y_test, proba_test_final)
high_recall_idx = np.where(recall >= 0.90)[0]
if len(high_recall_idx) > 0:
    # Pick the one with highest precision among those with 90%+ recall
    best_idx = high_recall_idx[np.argmax(precision[high_recall_idx])]
    threshold_high_sensitivity = thresholds[best_idx]
else:
    threshold_high_sensitivity = 0.3

from sklearn.metrics import precision_score, recall_score, f1_score
pred_hs = (proba_test_final >= threshold_high_sensitivity).astype(int)
prec_hs = precision_score(y_test, pred_hs, zero_division=0)
rec_hs = recall_score(y_test, pred_hs, zero_division=0)
f1_hs = f1_score(y_test, pred_hs, zero_division=0)

print(f"  Threshold: {threshold_high_sensitivity:.3f}")
print(f"  Precision: {prec_hs:.3f}")
print(f"  Recall: {rec_hs:.3f}")
print(f"  F1: {f1_hs:.3f}")
print(f"  % flagged: {pred_hs.mean():.1%}")

# Strategy 2: High Precision (Top-K targeting for commercial)
print("\nStrategy 2: HIGH PRECISION (Commercial Targeting)")
print("  Goal: Target top 10% with highest predicted probability")

threshold_top10 = np.percentile(proba_test_final, 90)
pred_hp = (proba_test_final >= threshold_top10).astype(int)
prec_hp = precision_score(y_test, pred_hp, zero_division=0)
rec_hp = recall_score(y_test, pred_hp, zero_division=0)
f1_hp = f1_score(y_test, pred_hp, zero_division=0)

print(f"  Threshold: {threshold_top10:.3f}")
print(f"  Precision: {prec_hp:.3f}")
print(f"  Recall: {rec_hp:.3f}")
print(f"  F1: {f1_hp:.3f}")
print(f"  % flagged: {pred_hp.mean():.1%} (by design)")

# Save thresholds
thresholds_df = pd.DataFrame({
    'strategy': ['High_Sensitivity', 'High_Precision'],
    'threshold': [threshold_high_sensitivity, threshold_top10],
    'precision': [prec_hs, prec_hp],
    'recall': [rec_hs, rec_hp],
    'f1': [f1_hs, f1_hp],
    'pct_flagged': [pred_hs.mean(), pred_hp.mean()],
    'use_case': ['Safety monitoring - catch all at-risk', 
                 'Commercial targeting - top decile']
})
thresholds_df.to_csv(OUT / "optimal_thresholds.csv", index=False)
print("\n✓ Saved: optimal_thresholds.csv")

# Confusion matrices for both strategies
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# High Sensitivity
cm_hs = confusion_matrix(y_test, pred_hs)
im1 = axes[0].imshow(cm_hs, cmap='Blues')
axes[0].set_xticks([0, 1])
axes[0].set_yticks([0, 1])
axes[0].set_xticklabels(['Pred Neg', 'Pred Pos'])
axes[0].set_yticklabels(['True Neg', 'True Pos'])
axes[0].set_title(f'High Sensitivity (Threshold={threshold_high_sensitivity:.3f})', 
                  fontweight='bold')
for i in range(2):
    for j in range(2):
        text = axes[0].text(j, i, f'{cm_hs[i, j]}\n({cm_hs[i, j]/cm_hs.sum():.1%})',
                           ha="center", va="center", color="black", fontsize=11)

# High Precision
cm_hp = confusion_matrix(y_test, pred_hp)
im2 = axes[1].imshow(cm_hp, cmap='Oranges')
axes[1].set_xticks([0, 1])
axes[1].set_yticks([0, 1])
axes[1].set_xticklabels(['Pred Neg', 'Pred Pos'])
axes[1].set_yticklabels(['True Neg', 'True Pos'])
axes[1].set_title(f'High Precision (Threshold={threshold_top10:.3f})', 
                  fontweight='bold')
for i in range(2):
    for j in range(2):
        text = axes[1].text(j, i, f'{cm_hp[i, j]}\n({cm_hp[i, j]/cm_hp.sum():.1%})',
                           ha="center", va="center", color="black", fontsize=11)

plt.tight_layout()
plt.savefig(OUT / "threshold_comparison.png", dpi=150)
plt.close()
print("✓ Saved: threshold_comparison.png")

# Apply both thresholds to full dataset
predictions['pred_high_sensitivity'] = (predictions['y_proba_calibrated'] >= threshold_high_sensitivity).astype(int)
predictions['pred_high_precision'] = (predictions['y_proba_calibrated'] >= threshold_top10).astype(int)

# Add risk categories with calibrated probs
predictions['risk_category'] = pd.cut(
    predictions['y_proba_calibrated'],
    bins=[0, 0.3, 0.7, 1.0],
    labels=['Low', 'Medium', 'High']
)

predictions.to_csv(OUT / "predictions_full_calibrated.csv", index=False)
print("✓ Updated predictions with both threshold strategies")

# ============================================================================
# C. SHAP EXPLANATIONS
# ============================================================================
print("\n" + "=" * 80)
print("C. SHAP EXPLANATIONS (Patient-Level)")
print("=" * 80)

try:
    import shap
    print("\nComputing SHAP values for RandomForest...")
    
    # Use TreeExplainer for RF
    estimator = best_model.named_steps['clf']
    explainer = shap.TreeExplainer(estimator)
    
    # Compute for test set
    shap_values = explainer.shap_values(X_test)
    
    # Handle multiclass output
    if isinstance(shap_values, list):
        sv = shap_values[1]  # Positive class
    else:
        sv = shap_values
    
    # Summary plot (bar)
    shap.summary_plot(sv, X_test, show=False, max_display=15, plot_type='bar')
    plt.tight_layout()
    plt.savefig(OUT / "shap_summary_bar.png", dpi=150, bbox_inches='tight')
    plt.close()
    print("✓ Saved: shap_summary_bar.png")
    
    # Summary plot (beeswarm)
    shap.summary_plot(sv, X_test, show=False, max_display=15)
    plt.tight_layout()
    plt.savefig(OUT / "shap_summary_beeswarm.png", dpi=150, bbox_inches='tight')
    plt.close()
    print("✓ Saved: shap_summary_beeswarm.png")
    
    # Waterfall plot for top 3 high-risk patients
    print("\nGenerating waterfall plots for top 3 high-risk patients...")
    
    # Get top 3 by predicted probability
    test_proba = best_model.predict_proba(X_test)[:, 1]
    top3_idx = np.argsort(test_proba)[-3:][::-1]
    
    for i, idx in enumerate(top3_idx):
        plt.figure(figsize=(10, 6))
        shap.waterfall_plot(
            shap.Explanation(
                values=sv[idx],
                base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
                data=X_test.iloc[idx],
                feature_names=X_test.columns.tolist()
            ),
            show=False,
            max_display=10
        )
        plt.tight_layout()
        plt.savefig(OUT / f"shap_waterfall_patient_{i+1}.png", dpi=150, bbox_inches='tight')
        plt.close()
    
    print(f"✓ Saved: shap_waterfall_patient_1/2/3.png")
    
    # Force plot for top patient (HTML)
    print("\nGenerating SHAP force plot (HTML)...")
    shap.force_plot(
        explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
        sv[top3_idx[0]],
        X_test.iloc[top3_idx[0]],
        matplotlib=False
    ).save_html(str(OUT / "shap_force_plot_top_patient.html"))
    print("✓ Saved: shap_force_plot_top_patient.html")
    
    print("\n✅ SHAP explanations complete")
    
except ImportError:
    print("\n⚠️  SHAP not installed")
    print("   Install with: pip install shap --break-system-packages")
    print("   Skipping SHAP generation...")
except Exception as e:
    print(f"\n⚠️  SHAP computation failed: {e}")
    print("   Continuing without SHAP explanations...")

# ============================================================================
# D. POWER BI READY DATASETS
# ============================================================================
print("\n" + "=" * 80)
print("D. POWER BI DATASETS")
print("=" * 80)

# 1. Main predictions table (already saved above with calibrated probs + thresholds)
print("\n✓ predictions_full_calibrated.csv")
print("  Columns: PatientID, y_true, y_proba, y_proba_calibrated,")
print("           pred_high_sensitivity, pred_high_precision, risk_category")

# 2. Thresholds reference table (already saved)
print("✓ optimal_thresholds.csv")
print("  Columns: strategy, threshold, precision, recall, f1, pct_flagged, use_case")

# 3. Feature importance (already exists from step 3)
if (OUT / "feature_importance_rf.csv").exists():
    print("✓ feature_importance_rf.csv (from step 3)")

# 4. Model scores (already exists from step 3)
if (OUT / "model_scores.csv").exists():
    print("✓ model_scores.csv (from step 3)")

# 5. Create risk distribution summary
risk_summary = predictions.groupby('risk_category').agg(
    count=('PatientID', 'count'),
    pct=('PatientID', lambda x: len(x) / len(predictions)),
    avg_proba=('y_proba_calibrated', 'mean'),
    actual_response_rate=('y_true', 'mean')
).reset_index()

risk_summary.to_csv(OUT / "risk_distribution.csv", index=False)
print("✓ risk_distribution.csv")
print("  Summary stats by risk category")

# 6. Create decile analysis
try:
    predictions['decile'] = pd.qcut(
        predictions['y_proba_calibrated'], 
        q=10, 
        labels=False,
        duplicates='drop'
    ) + 1  # 1-indexed deciles
    predictions['decile'] = 'D' + predictions['decile'].astype(str)
except ValueError:
    # If qcut fails, use percentile ranks
    predictions['decile'] = pd.cut(
        predictions['y_proba_calibrated'].rank(pct=True),
        bins=10,
        labels=[f'D{i}' for i in range(1, 11)]
    )

decile_analysis = predictions.groupby('decile').agg(
    n_patients=('PatientID', 'count'),
    avg_predicted_prob=('y_proba_calibrated', 'mean'),
    actual_response_rate=('y_true', 'mean'),
    min_prob=('y_proba_calibrated', 'min'),
    max_prob=('y_proba_calibrated', 'max')
).reset_index()

decile_analysis.to_csv(OUT / "decile_analysis.csv", index=False)
print("✓ decile_analysis.csv")
print("  Decile-level performance metrics")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "=" * 80)
print("PRODUCTION ARTIFACTS COMPLETE")
print("=" * 80)

print("\n📂 Files created for Power BI:")
print("\nModels:")
print("  • model_rf_calibrated.joblib")

print("\nDatasets:")
print("  • predictions_full_calibrated.csv (MAIN - use this)")
print("  • optimal_thresholds.csv")
print("  • risk_distribution.csv")
print("  • decile_analysis.csv")
print("  • feature_importance_rf.csv (from step 3)")
print("  • model_scores.csv (from step 3)")

print("\nVisualizations:")
print("  • calibration_curve.png")
print("  • threshold_comparison.png")
print("  • shap_summary_bar.png (if SHAP installed)")
print("  • shap_summary_beeswarm.png (if SHAP installed)")
print("  • shap_waterfall_patient_*.png (if SHAP installed)")
print("  • roc_curve_best.png (from step 3)")
print("  • pr_curve_best.png (from step 3)")

print("\n📊 Key Numbers for Laurent:")
print(f"  • Calibrated model AUC: Same as original (~0.94)")
print(f"  • High Sensitivity threshold: {threshold_high_sensitivity:.3f} (catches {rec_hs:.1%} of positives)")
print(f"  • High Precision threshold: {threshold_top10:.3f} (top 10%, precision {prec_hp:.1%})")
print(f"  • Total patients: {len(predictions)}")
print(f"  • High risk patients: {(predictions['risk_category']=='High').sum()} ({(predictions['risk_category']=='High').mean():.1%})")

print("\n✅ Ready for Power BI integration")
print("=" * 80)


In [1]:
# --- This is your existing code ---
patients_df = ... # Your code to create the features dataframe
predictions_df = ... # Your code to create the predictions dataframe

# --- Add these two new lines ---
print("Patients Features Columns:", patients_df.columns.tolist())
print("Predictions Columns:", predictions_df.columns.tolist())

# --- This is your existing code ---
patients_df.to_csv('data/patients_features.csv', index=False)
predictions_df.to_csv('data/predictions_full_bestmodel.csv', index=False)

AttributeError: 'ellipsis' object has no attribute 'columns'